# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [106]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [107]:
#menampilkan informasi dataset yang sudah diclustering
nel=pd.read_csv('Dataset_clustering.csv')
nel.head()

,Ship_Type,Route_Type,Engine_Type,Maintenance_Status,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Draft_meters,Weather_Condition,Cargo_Weight_tons,Operational_Cost_USD,Revenue_per_Voyage_USD,Turnaround_Time_hours,Weekly_Voyage_Count,Average_Load_Percentage,Cluster
0,Container Ship,Long-haul,Heavy Fuel Oil (HFO),Critical,12.597558,2062.983982,1030.943616,14.132284,Moderate,1959.017882,483832.354540,292183.273104,25.867077,1,93.769249,0
1,Fish Carrier,Short-haul,Steam Turbine,Good,10.387580,1796.057415,1060.486382,14.653083,Rough,162.394712,483388.000509,883765.787360,63.248196,6,93.895372,2
2,Container Ship,Long-haul,Diesel,Fair,20.749747,1648.556685,658.874144,7.199261,Moderate,178.040917,448543.404044,394018.746904,49.418150,9,96.218244,1
3,Bulk Carrier,Transoceanic,Steam Turbine,Fair,21.055102,915.261795,1126.822519,11.789063,Moderate,1737.385346,261349.605449,87551.375175,22.409110,1,66.193698,2
4,Fish Carrier,Transoceanic,Diesel,Fair,13.742777,1089.721803,1445.281159,9.727833,Moderate,260.595103,287718.375160,676121.459632,64.158231,8,80.008581,1


In [108]:
cluster_labels = nel['Cluster']
nel_encode = pd.get_dummies(nel.drop(columns=['Cluster']), columns=['Ship_Type', 'Engine_Type', 'Route_Type'])
nel_encode = nel_encode.replace({True: 1, False: 0})
label_encoder = LabelEncoder()
feature_ordinal = ['Weather_Condition', 'Maintenance_Status']
for feature in feature_ordinal:
    nel_encode[feature] = label_encoder.fit_transform(nel_encode[feature])
nel_encode['Cluster'] = cluster_labels
nel_encode.head()


<ipython-input-108-06c909a48d63>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  nel_encode = nel_encode.replace({True: 1, False: 0})


,Maintenance_Status,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Draft_meters,Weather_Condition,Cargo_Weight_tons,Operational_Cost_USD,Revenue_per_Voyage_USD,Turnaround_Time_hours,...,Ship_Type_Fish Carrier,Ship_Type_Tanker,Engine_Type_Diesel,Engine_Type_Heavy Fuel Oil (HFO),Engine_Type_Steam Turbine,Route_Type_Coastal,Route_Type_Long-haul,Route_Type_Short-haul,Route_Type_Transoceanic,Cluster
0,0,12.597558,2062.983982,1030.943616,14.132284,1,1959.017882,483832.354540,292183.273104,25.867077,...,0,0,0,1,0,0,1,0,0,0
1,2,10.387580,1796.057415,1060.486382,14.653083,2,162.394712,483388.000509,883765.787360,63.248196,...,1,0,0,0,1,0,0,1,0,2
2,1,20.749747,1648.556685,658.874144,7.199261,1,178.040917,448543.404044,394018.746904,49.418150,...,0,0,1,0,0,0,1,0,0,1
3,1,21.055102,915.261795,1126.822519,11.789063,1,1737.385346,261349.605449,87551.375175,22.409110,...,0,0,0,0,1,0,0,0,1,2
4,1,13.742777,1089.721803,1445.281159,9.727833,1,260.595103,287718.375160,676121.459632,64.158231,...,1,0,1,0,0,0,0,0,1,1


In [109]:
#Normalisasi data
scaler = MinMaxScaler()
nel_encode.iloc[:, :-1] = scaler.fit_transform(nel_encode.iloc[:, :-1])

<ipython-input-109-04be2b1f0a92>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.  1.  0.5 ... 0.  1.  0.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nel_encode.iloc[:, :-1] = scaler.fit_transform(nel_encode.iloc[:, :-1])
<ipython-input-109-04be2b1f0a92>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5 1.  0.5 ... 0.  0.5 0.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nel_encode.iloc[:, :-1] = scaler.fit_transform(nel_encode.iloc[:, :-1])
<ipython-input-109-04be2b1f0a92>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.    0.625 1.    ... 0.625 0.125 0.125]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nel_encode.iloc[:, :-1

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [110]:
X = nel_encode.drop('Cluster', axis=1)
y = nel_encode['Cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Jumlah data training: {X_train.shape[0]}")
print(f"Jumlah data testing: {X_test.shape[0]}")

Jumlah data training: 2188
Jumlah data testing: 548


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [111]:
# Menggunakan Algoritma Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [112]:
# Menggunakan Algoritma XGBoost
xgb_model = XGBClassifier(objective='multi:softmax', num_class=4, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:22:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=4, num_parallel_tree=None, ...)

In [113]:
#Menggunakan Algoritma KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [114]:
#Menggunakan Algoritma SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

SVC()

In [115]:
#Menggunakan Algoritma Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [116]:
#Menggunakan Algoritma Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB()

Dalam proyek ini, kita menggunakan empat algoritma klasifikasi: **Random Forest, XGBoost, K-Nearest Neighbors (KNN), dan Support Vector Machine (SVM).**

**Random Forest** menggabungkan banyak decision tree menggunakan teknik bagging untuk meningkatkan akurasi dan mengurangi overfitting. **XGBoost** menggunakan boosting untuk memperbaiki kesalahan secara bertahap, meningkatkan performa prediksi. **KNN** mengklasifikasikan data berdasarkan kedekatan dengan tetangga terdekat, sementara **SVM** mencari hyperplane optimal untuk memisahkan kelas dengan margin maksimal.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [79]:
#Prediksi dengan Algoritma Random Forest
y_pred_rf = rf_model.predict(X_test)

In [80]:
#Prediksi dengan Algoritma XGBoost
y_pred_xgb = xgb_model.predict(X_test)

In [96]:
#Prediksi dengan Algoritma KNN
y_pred_knn = knn_model.predict(X_test)

In [97]:
#Prediksi dengan Algoritma SVM
y_pred_svm = svm_model.predict(X_test)

In [117]:
#Prediksi dengan Algoritma Decision Tree
y_pred_dt = dt_model.predict(X_test)

In [118]:
#Prediksi dengan Algoritma Naive Bayes
y_pred_nb = nb_model.predict(X_test)

In [81]:
#Menghitung matrik evaluasi Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

In [82]:
#Menghitung matrik evaluasi XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')

In [98]:
#Menghitung matrix evaluasi KNN
accuracy_knn = accuracy_score(y_test, y_pred_knn)
precision_knn = precision_score(y_test, y_pred_knn, average='weighted')
recall_knn = recall_score(y_test, y_pred_knn, average='weighted')
f1_knn = f1_score(y_test, y_pred_knn, average='weighted')

In [99]:
#Menghitung matrix evaluasi SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm, average='weighted')
recall_svm = recall_score(y_test, y_pred_svm, average='weighted')
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')

In [119]:
#Menghitung matrix evaluasi Decision Tree
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt, average='weighted')
recall_dt = recall_score(y_test, y_pred_dt, average='weighted')
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')

In [120]:
#Menghitung matrix evaluasi Naive Bayes
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb, average='weighted')
recall_nb = recall_score(y_test, y_pred_nb, average='weighted')
f1_nb = f1_score(y_test, y_pred_nb, average='weighted')

In [83]:
#Menampilkan hasil evaluasi Random Forest
print("Evaluasi Random Forest:")
print(f"Accuracy: {accuracy_rf}")
print(f"Precision: {precision_rf}")
print(f"Recall: {recall_rf}")
print(f"F1-Score: {f1_rf}")


Evaluasi Random Forest:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [84]:
#Menampilkan hasil evaluasi XGBoost
print("Evaluasi XGBoost:")
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"Precision: {precision_xgb:.4f}")
print(f"Recall: {recall_xgb:.4f}")
print(f"F1-Score: {f1_xgb:.4f}")

Evaluasi XGBoost:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000


In [100]:
#Menampilkan hasil evaluasi KNN
print("Evaluasi KNN:")
print(f"Accuracy: {accuracy_knn}")
print(f"Precision: {precision_knn}")
print(f"Recall: {recall_knn}")
print(f"F1-Score: {f1_knn}")

Evaluasi KNN:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [101]:
#Menampilkan hasil evaluasi SVM
print("Evaluasi SVM:")
print(f"Accuracy: {accuracy_svm}")
print(f"Precision: {precision_svm}")
print(f"Recall: {recall_svm}")
print(f"F1-Score: {f1_svm}")

Evaluasi SVM:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [121]:
#Menampilkan hasil evaluasi Decision Tree
print("Evaluasi Decision Tree:")
print(f"Accuracy: {accuracy_dt}")
print(f"Precision: {precision_dt}")
print(f"Recall: {recall_dt}")
print(f"F1-Score: {f1_dt}")

Evaluasi Decision Tree:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [122]:
#Menampilkan hasil evaluasi Naive Bayes
print("Evaluasi Naive Bayes:")
print(f"Accuracy: {accuracy_nb}")
print(f"Precision: {precision_nb}")
print(f"Recall: {recall_nb}")
print(f"F1-Score: {f1_nb}")

Evaluasi Naive Bayes:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [85]:
#Menampilkan confusion matrix Random Forest
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
conf_matrix_rf

array([[157,   0,   0],
       [  0, 220,   0],
       [  0,   0, 171]])

Berdasarkan interpretasi confusion matrik jika dihitung dengan perhitungan dengan rumus yang disediakan, hasil untuk evaluasi Random Forest adalah benar dan jumlah elemen di matrix tersebut juga sesuai dengan total data testing sehingga output yang dikeluarkan harusnya sesuai.

In [86]:
#Menampilkan confusion matrix XGBoost
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
conf_matrix_xgb

array([[157,   0,   0],
       [  0, 220,   0],
       [  0,   0, 171]])

In [102]:
#Menampilkan confusion matrix KNN
conf_matrix_knn = confusion_matrix(y_test, y_pred_knn)
conf_matrix_knn

array([[157,   0,   0],
       [  0, 220,   0],
       [  0,   0, 171]])

In [103]:
#Menampilkan confusion matrix SVM
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
conf_matrix_svm

array([[157,   0,   0],
       [  0, 220,   0],
       [  0,   0, 171]])

In [123]:
#Menampilkan confusion matrix Decision Tree
conf_matrix_dt = confusion_matrix(y_test, y_pred_dt)
conf_matrix_dt

array([[157,   0,   0],
       [  0, 220,   0],
       [  0,   0, 171]])

In [124]:
#Menampilkan confusion matrix Naive Bayes
conf_matrix_nb = confusion_matrix(y_test, y_pred_nb)
conf_matrix_nb

array([[157,   0,   0],
       [  0, 220,   0],
       [  0,   0, 171]])

In [126]:
#Menampilkan tabel secara keseluruhan
models = {
    "Random Forest": rf_model,
    "XGBoost": xgb_model,
    "KNN": knn_model,
    "SVM": svm_model,
    "Decision Tree": dt_model,
    "Naive Bayes": nb_model
}
results = []

for name, model in models.items():
    y_pred = model.predict(X_test)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average='weighted'),
        "Recall": recall_score(y_test, y_pred, average='weighted'),
        "F1-Score": f1_score(y_test, y_pred, average='weighted')
    })
df_results = pd.DataFrame(results)
print(df_results)

           Model  Accuracy  Precision  Recall  F1-Score
0  Random Forest       1.0        1.0     1.0       1.0
1        XGBoost       1.0        1.0     1.0       1.0
2            KNN       1.0        1.0     1.0       1.0
3            SVM       1.0        1.0     1.0       1.0
4  Decision Tree       1.0        1.0     1.0       1.0
5    Naive Bayes       1.0        1.0     1.0       1.0


* Berdasarkan output diatas baik Random Forest, XGBoost, KNN, SVM, Decision Tree, & Naive Bayes menunjukkan hasil yang sempurna, dengan akurasi, precision, recall, dan F1-score mencapai 100%.
* Confusion matrix menunjukkan tidak ada kesalahan klasifikasi, yang berarti setiap sampel dalam data uji diklasifikasikan ke kelas yang benar.
* Hasil yang terlalu sempurna (100%) bisa mengindikasikan overfitting, terutama jika data uji memiliki distribusi yang sangat mirip dengan data latih.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

1.**Identifikasi Kelemahan Model**

Berdasarkan hasil evaluasi, semua model (Random Forest, XGBoost, KNN, SVM, Decision Tree, dan Naive Bayes) memperoleh akurasi, precision, recall, dan F1-score sebesar 100%.
Tidak ditemukan kelemahan signifikan seperti precision atau recall rendah untuk kelas tertentu, karena model mampu mengklasifikasikan semua sampel dengan sempurna.
Potensi penyebab hasil sempurna ini dapat dijelaskan sebagai berikut:
* Pola data yang sangat jelas, sehingga model dapat memisahkan kelas dengan sempurna.
* Fitur yang sangat deterministik, di mana atribut dalam dataset memiliki hubungan langsung dengan label.
* Tidak ada noise atau ketidakseimbangan data, sehingga model dapat belajar dengan optimal tanpa kesalahan klasifikasi.

2.**Apakah Model Mengalami Overfitting atau Underfitting?**
* Tidak ditemukan indikasi underfitting, karena model mampu mencapai performa optimal tanpa kesalahan klasifikasi.
* Potensi overfitting perlu diuji lebih lanjut, tetapi jika model tetap memberikan hasil sempurna pada data baru (unseen data), maka kemungkinan overfitting rendah.

Untuk memastikan hasil ini valid, beberapa langkah yang bisa dilakukan:
* Cek dengan data tambahan atau validasi silang (cross-validation).
* Pastikan tidak ada kebocoran data antara training dan testing.

3.**Rekomendasi Tindakan Lanjutan**
* Jika Datanya sudah sesuai, maka model ini sudah optimal dan siap digunakan.
* Jika ingin menguji ketahanan model, bisa dicoba dengan dataset yang lebih besar atau lebih kompleks.
* Jika ingin menghindari overfitting, bisa diterapkan regularisasi atau reduksi fitur.

**Kesimpulan:**
Model yang digunakan telah mencapai akurasi sempurna, tanpa indikasi kelemahan yang signifikan. Oleh karena itu, tidak diperlukan perbaikan lebih lanjut.
